### Clear memory

In [25]:
%reset -f
import gc
gc.collect()

0

### Import

In [26]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import pymupdf, json, requests, re, sys
from sentence_transformers import SentenceTransformer
from pathlib import Path
from tqdm.auto import tqdm
import langchain
from typing import List, Tuple, Dict, Any, Optional
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
sns.set_style('whitegrid')
%matplotlib inline

In [27]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [28]:
sys.path.append('..')
from src.data_utils import *

# Data Loading
Load chunks, prepared in the first notebook

In [ ]:
chunks = pd.read_json('../data/processed/chunks.json', orient='records')

chunks.head(3)

,chunk_id,text,page_num,char_count,start_char,end_char
0,0,User Guide AWS Toolkit for Microsoft Azure Dev...,1,134,0,134
1,1,AWS Toolkit for Microsoft Azure DevOps User Gu...,2,422,0,422
2,2,"s likely to cause confusion among customers, o...",2,260,322,822


In [ ]:
print('Number of chunks:', chunks.shape[0])

# ца